<a href="https://colab.research.google.com/github/drbilal216/AI_201/blob/main/rag_ontext_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependies

In [1]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.3 MB/s eta 0:00:00


In [2]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)

## Initialization
Before initializing our vector store, let's connect to a Pinecone index. If one named index_name doesn't exist, it will be created.

dimension of embeddings


depend on the model we are using for embedding we will use this number
eg
open ai
google

In [3]:
import time

index_name = "langchain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768, # Replace with your model dimensions # google = 768 # 3072 ?
        metric="cosine", # Replace with your model metric
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [4]:
print(index)

### We also need a pretrained text to embedding genrator

https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/task-types

In [5]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",
                                          google_api_key=GOOGLE_API_KEY)

In [7]:
# vector = embeddings.embed_query("hello, world!")
# vector[:5]

In [ ]:
# we need these for pinecone
# index
# embeddings

# pdf loader

In [ ]:
%pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

#file_path
file_b = "b.pdf"
file_m = "m.pdf"

In [ ]:
loader = PyPDFLoader(file_b)
cv1 = []
async for page in loader.alazy_load():
    cv1.append(page)

print(f"{cv1[0].metadata}\n")
# print(cv1[0].page_content)

{'source': 'b.pdf', 'page': 0, 'page_label': '1'}



In [ ]:
loader = PyPDFLoader(file_m)
cv2 = []
async for page in loader.alazy_load():
    cv2.append(page)

print(f"{cv2[0].metadata}\n")
# print(cv2[0].page_content)

{'source': 'm.pdf', 'page': 0, 'page_label': '1'}



### Pinecone Vectorstore
Now that our Pinecone index is setup, we can initialize our vector store.

In [8]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

### add items to vector store

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content=(cv1[0].page_content),
    metadata=(cv1[0].metadata),
)

document_2 = Document(
    page_content=(cv2[0].page_content),
    metadata=(cv2[0].metadata),
)

In [ ]:
documents = [document_1, document_2]

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

# uuids = [str(uuid4()) for _ in range(len(documents))]
uuids = ["1", "2"]

In [ ]:
# upload to vector db pine cone

vector_store.add_documents(documents=documents, ids=uuids)

['1', '2']

In [ ]:
#Delete items from vector store
# vector_store.delete(ids=[uuids[-1]])

# data retreival


In [11]:
# first way

results = vector_store.similarity_search(
    "mustafa",
    k=1, # data to retreive
    # filter={"source": "tweet"}, #
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Research Papers - Strategist/ Coded
GANs over distributed SystemBS in Software Engineering
BSE - Bahria University 2020-2024
EDUCATION
Microsoft Certified: Python Exam(98-381)
PIAIC Certified: AI Application Developer
CERTIFICATIONS
Backend Developer with expertise in SQL, MongoDB, Node.js, and FastAPI. Improved server response times
and managed databases. Enhanced user engagement through integrating OpenAI APIs and developed
secure smart contracts with Solidity
FULLSTACK AI ENGINEER / SOFTWARE ENGINEER
+92-336-3739689 · mustafatola02@gmail.com 
Hussainabad, Karachi, Pakistan
Mustufa 
Machine Learning/ Deep learning - Keras, Tensorflow
Data Analysis - Seaborn, Matplotlib, Pandas
Data Apps - Streamlit
Automation/ Integrations - Selenium, Make.com
Database - SQL, MongoDB
GENERAL SKILLS
Speed Programming - Teknofest Pakistan
Took 8th position
COMPETITIONS
Juniper Networks Global AI Challenge
7th Position - AI Planet
Backend Developer - Metafourt Technologies
Made backend system of freel

In [18]:
# second way

results = vector_store.similarity_search_with_score(
    "What is bilal age",
    k=2,
    # filter={"source": "news"}
)
for res, score in results:
    # print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")
    print(f"{score:3f}")
    print(f"{res.page_content}")
    print("--------------------------")

0.585297
AI ENGINEER@ THE DISRUPT LABS, PAKISTAN
Bilal Muhammad Khan
PROFESSIONAL SUMMARY
I am an AI engineer specializing in developing advanced algorithms for image processing and deep learning.
I have 1.5 years of experience working with computer vision applications.
PROJECTS
Movie Recommendation System - Machine Learning 
Implemented K-Nearest Neighbors Algorithm(KNN) on MovieLens Dataset for quick Movie
Recommendation and made 2 models, User Based and Item Based. 
Natural Language Processing (NLP) - Deep Learning 
Applied advanced deep learning techniques, including Recurrent Neural Networks (RNN), Long
Short-Term Memory (LSTM), and Gated Recurrent Unit (GRU), with word embedding, to IMDb
Movie Reviews dataset for sentiment analysis.
House Price Prediction (Regression) - Deep Learning 
Proficient in regression analysis using deep learning techniques, including application to datasets
such as the Boston Housing Dataset. 
AI-Based Monitoring System - Yunus textile, Pakistan
Tracks v

### response with LLM

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    google_api_key=GOOGLE_API_KEY
    )

In [22]:
def answer_to_user(query: str):
  # Vector_Search
  vector_results = vector_store.similarity_search(query, k=1)
  # Pass to Model Vector result + querry
  final_answer = llm.invoke(f"ANSWER THIS USER QUERRY: {query}, HERE ARE SOME REFERENCES: {vector_results}")
  return final_answer

In [25]:
response = answer_to_user("What are bilal skillset?")

In [26]:
print(response.content)

Based on the provided document, Bilal's skillset includes:

**Core AI & Machine Learning Skills:**

*   **AI Engineering:**  Specializes in developing advanced algorithms for image processing and deep learning.
*   **Machine Learning:** Experience with algorithms like K-Nearest Neighbors (KNN).
*   **Deep Learning:** Proficient in using deep learning techniques for various tasks, including regression, sentiment analysis, and object detection.
*   **Natural Language Processing (NLP):** Experience applying deep learning techniques like RNN, LSTM, and GRU with word embeddings for sentiment analysis.
*   **Regression Analysis:** Skilled in using deep learning for regression tasks.
*   **Object Detection:**  Experience with custom YOLO object detection models.
*   **Object Tracking:** Worked with object tracking techniques.
*   **Pose Estimation:**  Experience with pose estimation techniques.

**Computer Vision Skills:**

*   **Image Processing:**  Strong background in image processing tech